In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [829 kB]
Hit:13 http://ppa.laun

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-03 20:55:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2021-12-03 20:55:56 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_01.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [9]:
# the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful 
vine_table_20 = df.filter("total_votes >=  20")

vine_table_20.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52534548|R18VIM840CEFRP|1621360075|     143884185|The Real Kosher J...|           Books|          1|           16|        105|   N|                N|                 SLH|I'm sorry, but ca...| 2012-05-03|
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|    

In [12]:
# Filter the new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

vine_table_50 = vine_table_20.filter("helpful_votes/total_votes >=0.5")

vine_table_50.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|
|         US|   52978967|R1363VA3TPNLVB|0670022969|     432940103|American Nations:...|           Books|          5|    

In [14]:
# create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid)
vine_table_Y = vine_table_50.filter("vine == 'Y'")

vine_table_Y.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50673938|R2Z8083V8ZFQVZ|1612183549|     561835180|Redemption (The P...|           Books|          3|           82|         87|   Y|                N|     Promising debut|The positives:  t...| 2012-05-02|
|         US|   35653914| RK25TAO4GGS5G|0547567375|     266689242|The Bell Bandit (...|           Books|          3|    

In [15]:
# create a new DataFrame that retrieves all the rows where a review was written as not part of the Vine program (unpaid)
vine_table_N = vine_table_50.filter("vine == 'N'")

vine_table_N.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|
|         US|   52978967|R1363VA3TPNLVB|0670022969|     432940103|American Nations:...|           Books|          5|    

In [16]:
# create a new DataFrame that has 5 star reviews for Y
vine_table_Y_5 = vine_table_Y.filter("star_rating == 5")

vine_table_Y_5.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15737193|R12CHC0CB2WASU|1612183174|     198757847|           King City|           Books|          5|           43|         48|   Y|                N|There's a new she...|Detective Tom Wad...| 2012-05-02|
|         US|   47770260|R27UEUDUYPLXZC|0132944677|     855208343|The College Solut...|           Books|          5|    

In [17]:
# create a new DataFrame that has 5 star reviews for N
vine_table_N_5 = vine_table_N.filter("star_rating == 5")

vine_table_N_5.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52978967|R1363VA3TPNLVB|0670022969|     432940103|American Nations:...|           Books|          5|          370|        388|   N|                Y|Like scales falli...|Colin Woodard has...| 2012-05-03|
|         US|   52445438| RBQZC5A3TSWT5|085768986X|      69886969|Joss Whedon: The ...|           Books|          5|    

In [18]:
# create a new DataFrame that has less than 5 star reviews for Y
vine_table_Y_lessthan5 = vine_table_Y.filter("star_rating != 5")

vine_table_Y_lessthan5.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50673938|R2Z8083V8ZFQVZ|1612183549|     561835180|Redemption (The P...|           Books|          3|           82|         87|   Y|                N|     Promising debut|The positives:  t...| 2012-05-02|
|         US|   35653914| RK25TAO4GGS5G|0547567375|     266689242|The Bell Bandit (...|           Books|          3|    

In [19]:
# create a new DataFrame that has less than 5 star reviews for N
vine_table_N_lessthan5 = vine_table_N.filter("star_rating != 5")

vine_table_N_lessthan5.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|
|         US|   52977415|R3OW0AIVLEDIQ7|0262016729|     624789283|The Clash of Gene...|           Books|          3|    

In [21]:

v_20 = vine_table_20.count()
v_50 = vine_table_50.count()
v_Y = vine_table_Y.count()
v_N = vine_table_N.count()
v_Y_5 = vine_table_Y_5.count()
v_N_5 = vine_table_N_5.count()
v_Y_lessthan5 = vine_table_Y_lessthan5.count()
v_N_lessthan5 = vine_table_N_lessthan5.count()

In [28]:
print("Total number of paid reviews %f" % v_Y)

print("Total number of unpaid reviews %f" % v_N)

print("Total number of paid 5 star reviews %f" % v_Y_5) 

print("Total number of unpaid 5 star reviews %f" % v_N_5)  

print("Percentage of paid 5 star reviews %f" % (v_Y_5/v_Y *100), "%")

print("Percentage of unpaid 5 star reviews %f" % (v_N_5/v_N *100), "%") 


Total number of paid reviews 4781.000000
Total number of unpaid reviews 332395.000000
Total number of paid 5 star reviews 1604.000000
Total number of unpaid 5 star reviews 168800.000000
Percentage of paid 5 star reviews 33.549467 %
Percentage of unpaid 5 star reviews 50.782954 %


In [29]:
print("Percentage of paid less than 5 star reviews %f" % (v_Y_lessthan5/v_Y *100), "%")

print("Percentage of unpaid less than 5 star reviews %f" % (v_N_lessthan5/v_N *100), "%") 

Percentage of paid less than 5 star reviews 66.450533 %
Percentage of unpaid less than 5 star reviews 49.217046 %
